TODO:
 ✔ fix that
 ✔ implement metagene
 - summary draft radar chart + table

 - ligation bias needs to be adjusted to global dinucleotide proportion
   + option for both local (current read only) or global
   + score = [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)
 - SEE CONFIG
   + distance_target: "start" #Options: ["start", "stop"] # Add both? grid plot
 ✔ add y-range limit
 ✔ nucleotide proportion (logoplot)
   ✔ dynamic image scaling for weblogo
 ✔ heatmap
   ✔ fix heatmap stretched cells
 - option for both ends ligation bias
 ✔ option for both ends metagene plots
 ( - offset calc)
 ( - sample-data)
 - check list (awesome riboseq)
 - tests
 - timing (factory/timeit)
 - SCORING: each module needs scores
 ✔ Machine-readable format (JSON/CSV(s))
  + More efficient data usage? (currently 121 MB for json file!) Culling data where possible?


In [2]:
from modules import *
from plots import *
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import yaml
import re
import gffpandas.gffpandas as gffpd
from file_parser import parse_bam
import tempfile
import subprocess
import base64

In [3]:
with open("../config.yml", "r") as ymlfile:
        config = yaml.load(ymlfile, Loader=yaml.Loader)
# config

In [4]:
# for new read_df, use parse_to_csv.py
read_df_pre = pd.read_csv("/home/lukas/projects/RiboMetric/test_files/sample_read_df_pre_1000000.csv")
annotation_df = pd.read_csv("/home/lukas/projects/RiboMetric/gencode_RibosomeProfiler.tsv",delimiter="\t")

In [5]:

read_df = read_df_pre.loc[read_df_pre.index.repeat(read_df_pre['count'])].reset_index(drop=True)
read_df = a_site_calculation(read_df)
annotated_read_df = annotate_reads(read_df, annotation_df)

In [15]:
metagene_profile_dict = metagene_profile(annotated_read_df)
metagene_profile_dict_end = metagene_profile(annotated_read_df, "stop")

In [9]:
plot_metagene_profile_dict = plot_metagene_profile(metagene_profile_dict, config)
plot_metagene_heatmap_dict = plot_metagene_heatmap(metagene_profile_dict, config)

In [ ]:
██████╗  ██╗ ██████╗  ██████╗ ███████╗ ██████╗ ███╗   ███╗███████╗
██╔══██╗ ██║ ██╔══██╗██╔═══██╗██╔════╝██╔═══██╗████╗ ████║██╔════╝
██████╔╝ ██║ ██████╔╝██║   ██║███████╗██║   ██║██╔████╔██║█████╗
██╔══██╗ ██║ ██╔══██╗██║   ██║╚════██║██║   ██║██║╚██╔╝██║██╔══╝
██║  ██║ ██║ ██████╔╝╚██████╔╝███████║╚██████╔╝██║ ╚═╝ ██║███████╗
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝ ╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝

   ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗   
   ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝   
   ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
   ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║   
   ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║   
   ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   

In [ ]:
██████╗  ██╗ ██████╗  ██████╗  ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗  
██╔══██╗ ██║ ██╔══██╗██╔═══██╗ ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝  
██████╔╝ ██║ ██████╔╝██║   ██║ ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
██╔══██╗ ██║ ██╔══██╗██║   ██║ ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║
██║  ██║ ██║ ██████╔╝╚██████╔╝ ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   

In [165]:
from rich.console import Console
from rich.text import Text
logo = Text(
    """
██████╗  ██╗ ██████╗  ██████╗ ███████╗ ██████╗ ███╗   ███╗███████╗
██╔══██╗ ██║ ██╔══██╗██╔═══██╗██╔════╝██╔═══██╗████╗ ████║██╔════╝
██████╔╝ ██║ ██████╔╝██║   ██║███████╗██║   ██║██╔████╔██║█████╗
██╔══██╗ ██║ ██╔══██╗██║   ██║╚════██║██║   ██║██║╚██╔╝██║██╔══╝
██║  ██║ ██║ ██████╔╝╚██████╔╝███████║╚██████╔╝██║ ╚═╝ ██║███████╗
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝ ╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝
""",
    style="bold blue",
)
logo += Text(
    """
   ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗   
   ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝   
   ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
   ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║   
   ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║   
   ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   
""",
    style="bold red",
)
# LINE 1
logo = Text(
    """
    ██████╗  ██╗ ██████╗  ██████╗  """,
    style="bold blue",end=""
)
logo += Text(
    """███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗ """,
    style="bold red",end=""
)
# LINE 2
logo += Text(
    """
    ██╔══██╗ ██║ ██╔══██╗██╔═══██╗ """,
    style="bold blue",
)
logo += Text(
    """████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝""",
    style="bold red",end=""
)
# LINE 3
logo += Text(
    """
    ██████╔╝ ██║ ██████╔╝██║   ██║ """,
    style="bold blue",
)
logo += Text(
    """██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗""",
    style="bold red",end=""
)
# LINE 4
logo += Text(
    """
    ██╔══██╗ ██║ ██╔══██╗██║   ██║ """,
    style="bold blue",
)
logo += Text(
    """██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║""",
    style="bold red",end=""
)
# LINE 5
logo += Text(
    """
    ██║  ██║ ██║ ██████╔╝╚██████╔╝ """,
    style="bold blue",
)
logo += Text(
    """██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║""",
    style="bold red",end=""
)
# LINE 6
logo += Text(
    """
    ╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  """,
    style="bold blue",
)
logo += Text(
    """╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝""",
    style="bold red",end=""
)
Console().print(logo)

    ██████╗  ██╗ ██████╗  ██████╗  ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗ 
    ██╔══██╗ ██║ ██╔══██╗██╔═══██╗ ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝
    ██████╔╝ ██║ ██████╔╝██║   ██║ ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗
    ██╔══██╗ ██║ ██╔══██╗██║   ██║ ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║
    ██║  ██║ ██║ ██████╔╝╚██████╔╝ ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║
    ╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝

In [133]:
count = 0
colnums = 1
frame_colors = {0:"#636efa", 1:"#ef553b", 2:"#00cc96"}
if metagene_profile_dict["start"] != {}:
    target_loop = ["start"]
    if metagene_profile_dict["stop"] != {}:
        target_loop.append("stop")
        colnums = 2
else:
    target_loop = ["stop"]


fig = make_subplots(rows=1, cols=colnums, shared_yaxes=False,
    subplot_titles=["Distance from 5'","Distance from 3'"] if len(target_loop) > 1 
    else ["Distance from 5'"] if target_loop == ["start"]
    else ["Distance from 3'"],
    )
for current_target in target_loop:
    count += 1
    metagene_dict = {}
    for inner_dict in metagene_profile_dict[current_target].values():
        for inner_key, inner_value in inner_dict.items():
            if inner_key in metagene_dict and metagene_dict[inner_key] != None:
                metagene_dict[inner_key] += inner_value if inner_value is not None else 0
            else:
                metagene_dict[inner_key] = inner_value if inner_value is not None else 0
    n = 0
    color = [(x%3) for x in metagene_dict.keys()]
    for i in color:
        color[n] = frame_colors[i]
        n += 1
        
    fig.add_trace(
        go.Bar(x=list(metagene_dict.keys()), y=list(metagene_dict.values()), 
            name="Distance from 5'" if current_target == "start" else "Distance from 3'",
            marker=dict(color=color)),
            row=1, col=count, 
    )

fig.update_layout(
    title="Metagene Profile",
    xaxis_title="Relative position",
    yaxis_title="Read Count",
    font=dict(
        family=config["plots"]["font_family"],
        size=18,
        color=config["plots"]["base_color"],
    ),
    bargap=0,
    showlegend=False,
)
if colnums > 1:
    fig.update_layout(
            xaxis=dict(domain=[0, 0.48], zeroline=False),  # Adjust domain and remove x-axis zeroline for subplot 1
            xaxis2=dict(domain=[0.52, 1], zeroline=False),  # Adjust domain and remove x-axis zeroline for subplot 2
    )
else:
    fig.update_layout(
            xaxis=dict(domain=[0, 1], zeroline=False),
    )

fig.update_xaxes(
    range=config["plots"]["metagene_profile"]["distance_range"]
)

In [136]:
count = 0
colnums = 1
if metagene_profile_dict["start"] != {}:
    target_loop = ["start"]
    if metagene_profile_dict["stop"] != {}:
        target_loop.append("stop")
        colnums = 2
else:
    target_loop = ["stop"]

fig = make_subplots(rows=1, cols=colnums, shared_yaxes=True,
    subplot_titles=["Distance from 5'","Distance from 3'"] if len(target_loop) > 1 
    else ["Distance from 5'"] if target_loop == ["start"]
    else ["Distance from 3'"],
    )

for current_target in target_loop:
    count += 1
    x_data = []
    y_data = []
    z_data = []
    for k1, v1 in metagene_profile_dict[current_target].items():
        for k2, v2 in v1.items():
            x_data.append(k2)
            y_data.append(k1)
            z_data.append(v2)

    fig.add_trace(
        go.Heatmap(
            x=x_data,
            y=y_data,
            z=z_data,
            colorscale=config["plots"]["metagene_profile"]["colorscale"],
            zmin=0,
            zmax=config["plots"]["metagene_profile"]["max_colorscale"],
        ),
        row=1, col=count,
    )
fig.update_xaxes(
    range=config["plots"]["metagene_profile"]["distance_range"]
    )
fig.update_layout(
    title="Metagene Heatmap",
    xaxis_title="Relative position (nt)",
    yaxis_title="Read length",
    font=dict(
        family=config["plots"]["font_family"],
        size=18,
        color=config["plots"]["base_color"],
    ),
    legend={"traceorder": "normal"},
    showlegend=False,
)
if colnums > 1:
    fig.update_layout(
            xaxis=dict(domain=[0, 0.48], zeroline=False),  # Adjust domain and remove x-axis zeroline for subplot 1
            xaxis2=dict(domain=[0.52, 1], zeroline=False),  # Adjust domain and remove x-axis zeroline for subplot 2
    )
else:
    fig.update_layout(
            xaxis=dict(domain=[0, 1], zeroline=False),
    )
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatterpolar(
  r=[1, 5, 2, 2, 3],
  theta=['processing cost','mechanical properties','chemical stability', 'thermal stability',
           'device integration'],
  fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()